In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from mpl_toolkits.mplot3d import Axes3D
import netCDF4
import numpy
import os
import pandas
from PIL import Image
import random
import readline
import scipy
from scipy import fft
from scipy import linalg
from scipy import stats
from scipy.stats import poisson, ttest_ind
import seaborn
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import normalize
from statsmodels.tsa.ar_model import AutoReg
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExTraTrack/rcp_files/'

In [3]:
# Open File
def Create_DF(File):
    Data = open(File, 'r')
    Rows = []
#
# Organize Data
    for Line in Data:
        Rows.append(Line.strip())
    Storm_Code = []
    Storm_List_Orig = []
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            Code = Rows[i][41:45]
            Storm_List_Orig.append(Code)
        else:
            Storm_Code.append(Code)
    Array = numpy.zeros((13, len(Rows)-len(Storm_List_Orig)))
    Time = []
    k = -1
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            k += 1
        else:
            l = len(Rows[i]) - 100
            Array[0][i-k-1] = float(Rows[i][0:6+l])
            Array[1][i-k-1] = float(Rows[i][9+l:14+l])
            Array[2][i-k-1] = float(Rows[i][17+l:24+l])
            Array[3][i-k-1] = float(Rows[i][26+l:31+l])
            Array[4][i-k-1] = float(Rows[i][34+l:41+l])
            Array[5][i-k-1] = float(Rows[i][44+l:51+l])
            Array[6][i-k-1] = float(Rows[i][54+l:61+l])
            Array[7][i-k-1] = float(Rows[i][64+l:71+l])
            Array[8][i-k-1] = float(Rows[i][74+l:81+l])
            Time.append(datetime.datetime(year=int(Rows[i][84+l:88+l]), month=int(Rows[i][90+l:92+l]), \
            day=int(Rows[i][94+l:96+l]), hour=int(Rows[i][98+l:100+l])))
#
# Create DataFrame to Store Data
    DF_All = pandas.DataFrame({"Orig Code": Storm_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "Dist(m)": Array[4], "Angle": Array[5], "B": Array[6], "VLT": Array[7], "VUT": Array[8], \
    "Orig Time(Z)": Time})
#
# Remove -999.0 Data Points
    DF = DF_All[(DF_All["B"] > -728) & (DF_All["VLT"] > -728) & (DF_All["VUT"] > -728)].reset_index()
    return (DF, Storm_List_Orig)

In [43]:
def Update_DF_Code(DF, Orig_Code, Year_Diff_A, Year, Count):
    DF_Storm = DF[DF["Orig Code"] == Orig_Code].reset_index()
    Time_Year = DF_Storm["Orig Time(Z)"][0].year + Year_Diff_A
    if Time_Year == Year:
        Count += 1
    else:
        Year += 1
        Count = 1
    if Count < 10:
        Code = str(Year)+"0"+str(Count)
    else:
        Code = str(Year)+str(Count)
    return (Code, Year, Count)

In [44]:
def Combine_Lists(DF, Orig_List, New_List, Year_Diff, New_Code, New_Time, ABC, Orig_Combine, New_Combine, Y):
    for i in range(len(DF)):
        for l in range(len(Orig_List)):
# Update Storm Code
            if DF["Orig Code"][i] == Orig_List[l]:
                New_Code.append(New_List[l])
# Update Time
        Orig_Time = DF["Orig Time(Z)"][i]
        Year_Update = Orig_Time.year + Year_Diff
# Febuary 29 Problems
        if Orig_Time.month == 2 and Orig_Time.day == 29:
            New_Time.append(DF["Orig Time(Z)"][i].replace(year=Year_Update, day=28))
        else:
            New_Time.append(DF["Orig Time(Z)"][i].replace(year=Year_Update))
    for m in range(len(Orig_List)):
        ABC.append(Y)
        Orig_Combine.append(Orig_List[m])
        New_Combine.append(New_List[m])
    return (New_Code, New_Time, ABC, Orig_Combine, New_Combine)

In [45]:
# Combine DataFrames
def Combine_DF(DF_A, DF_B, DF_C, Orig_List_A, Orig_List_B, Orig_List_C, Model):
    if Model == "Control":
        Year_Start = 1900
#        Year_Diff_A = Year_Start - 1985
#        Year_Diff_B = Year_Start - 1985 + 30
#        Year_Diff_C = Year_Start - 1985 + 60
    New_List_A = []
    New_List_B = []
    New_List_C = []
    New_Time = []
    New_Code = []
    ABC = []
    Orig_Combine = []
    New_Combine = []
# Dataset A
    Year = Year_Start
    Year_Diff_A = Year - DF_A["Orig Time(Z)"][0].year
    Count = 0
    for k in range(len(Orig_List_A)):
        Code, Year, Count = Update_DF_Code(DF_A, Orig_List_A[k], Year_Diff_A, Year, Count)
        New_List_A.append(Code)
# Dataset B
    Year += 1
    Year_Diff_B = Year - DF_B["Orig Time(Z)"][0].year
    Count = 0
    for k in range(len(Orig_List_B)):
        Code, Year, Count = Update_DF_Code(DF_B, Orig_List_B[k], Year_Diff_B, Year, Count)
        New_List_B.append(Code)
# Dataset C
    Year += 1
    Year_Diff_C = Year - DF_C["Orig Time(Z)"][0].year
    Count = 0
    for k in range(len(Orig_List_B)):
        Code, Year, Count = Update_DF_Code(DF_C, Orig_List_C[k], Year_Diff_C, Year, Count)
        New_List_C.append(Code)
#
# Combine Into New Time and Codes List
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_A, Orig_List_A, New_List_A, Year_Diff_A, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "A")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_B, Orig_List_B, New_List_B, Year_Diff_B, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "B")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_C, Orig_List_C, New_List_C, Year_Diff_C, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "C")
#
# Concatenate Other Variables
    Vars = ["Lon", "Lat", "SLP(hPa)", "Winds(m/s)", "Dist(m)", "Angle", "B", "VLT", "VUT"]
    Total_Length = len(New_Code)
    Array = numpy.zeros((9,Total_Length))
    for n in range(len(Vars)):
        Array[n] = numpy.concatenate((DF_A[Vars[n]], DF_B[Vars[n]], DF_C[Vars[n]]))
#
# Create DataFrame to Store Data
    DF_Combine = pandas.DataFrame({"New Code": New_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "Dist(m)": Array[4], "Angle": Array[5], "B": Array[6], "VLT": Array[7], "VUT": Array[8], \
    "Time(Z)": New_Time})
    Codes_DF = pandas.DataFrame({"ABC": ABC, "Orig Code": Orig_Combine, "New Code": New_Combine})
    return (DF_Combine, Codes_DF)

In [46]:
# Open File
Control_A_DF_Orig, Control_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.002_avg')
Control_DF, Control_Codes = Combine_DF(Control_A_DF_Orig, Control_A_DF_Orig, Control_A_DF_Orig, \
Control_A_Storm_List_Orig, Control_A_Storm_List_Orig, Control_A_Storm_List_Orig, "Control")
Control_DF

,New Code,Lon,Lat,SLP(hPa),Winds(m/s),Dist(m),Angle,B,VLT,VUT,Time(Z)
0,190001,-82.19,27.99,1015.45,13.9,-999.00,-999.00,4.71,37.04,-36.64,1900-06-19 12:00:00
1,190001,-82.26,27.79,1015.45,13.8,23.61,197.20,6.18,-12.97,-23.15,1900-06-19 18:00:00
2,190001,-82.33,27.58,1015.45,13.7,23.61,197.23,3.28,-8.22,-18.60,1900-06-20 00:00:00
3,190001,-82.40,27.38,1015.45,13.7,23.62,197.27,3.45,-0.38,-18.54,1900-06-20 06:00:00
4,190001,-82.46,28.31,1014.22,11.7,103.18,356.87,1.72,8.23,-4.21,1900-06-20 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...
32824,198908,-77.88,40.72,998.40,12.5,341.66,317.64,83.26,-199.49,-198.54,1989-11-12 18:00:00
32825,198908,-80.75,42.25,1001.40,18.5,293.74,306.02,83.03,-293.51,-173.83,1989-11-13 00:00:00
32826,198908,-81.75,42.00,1001.48,14.8,87.03,251.42,73.58,-310.52,-142.11,1989-11-13 06:00:00
32827,198908,-82.50,42.00,1001.89,12.5,61.98,270.00,62.77,-296.47,-140.77,1989-11-13 12:00:00


In [47]:
Control_Codes

,ABC,Orig Code,New Code
0,A,0024,190001
1,A,0027,190002
2,A,0029,190003
3,A,0046,190004
4,A,0048,190005
...,...,...,...
940,C,1512,198904
941,C,1516,198905
942,C,1520,198906
943,C,1521,198907


In [ ]:
# Open File
def Create_DF_2(File, Control_Codes):
    Data = open(File, 'r')
    Rows = []
#
# Organize Data
    for Line in Data:
        Rows.append(Line.strip())
#    print (Rows)
    Storm_Code = []
    Start_Time = []
    End_Time = []
    Array = numpy.zeros((4, len(Rows)))
    for i in range(len(Rows)):
#        print (Rows[i][53:57])
        Storm_Code.append(str(Rows[i][0:4]))
        Array[0][i] = int(Rows[i][7:10]) * 6
        Array[1][i] = int(Rows[i][12])
        Array[2][i] = float(Rows[i][15:22])
        Array[3][i] = float(Rows[i][23:30])
        Start_Time.append(datetime.datetime(year=int(Rows[i][33:37]), month=int(Rows[i][39:41]), \
        day=int(Rows[i][43:45]), hour=int(Rows[i][47:49])))
        End_Time.append(datetime.datetime(year=int(Rows[i][53:57]), month=int(Rows[i][59:61]), \
        day=int(Rows[i][63:65]), hour=int(Rows[i][67:69])))
#
# Create DataFrame to Store Data
    DF = pandas.DataFrame({"Code": Storm_Code, "Path Type": Array[1], "Start Time": Start_Time, "End Time": End_Time, \
    "ET Duration (hr)": Array[0], "Start SLP": Array[2], "End SLP": Array[3]})
    return (DF)